In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings ("ignore")


import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re

import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle
from tqdm import tqdm
import os




# 1. Reading Data

In [2]:
path = r"F:\\study\\dataset\\amazon_food_reviews\\"
con = sqlite3.connect (path + 'database.sqlite')

In [3]:
filtered_data = pd.read_sql_query ("""
                                   SELECT *
                                   FROM Reviews 
                                   WHERE Score != 3 
                                   LIMIT 5000""", con)

In [4]:
def partition (x):
    if x < 3:
        return 0
    else:
        return 1

In [5]:
actual_score = filtered_data ["Score"]
positivenegative = actual_score.map (partition)
filtered_data ['Score'] = positivenegative
print ("Number of datapoints in our data", filtered_data.shape)
filtered_data.head (3)

Number of datapoints in our data (5000, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


In [6]:
display = pd.read_sql_query ("""
SELECT UserId, ProductId, ProfileName, Time, Score, Text, COUNT (*)
FROM Reviews
GROUP BY UserId
Having COUNT (*) >1
""", con)

In [7]:
print (display.shape)
display.head ()

(80668, 7)


,UserId,ProductId,ProfileName,Time,Score,Text,COUNT (*)
0,#oc-R115TNMSPFT9I7,B005ZBZLT4,Breyton,1331510400,2,Overall its just OK when considering the price...,2
1,#oc-R11D9D7SHXIJB9,B005HG9ESG,"Louis E. Emory ""hoppy""",1342396800,5,"My wife has recurring extreme muscle spasms, u...",3
2,#oc-R11DNU2NBKQ23Z,B005ZBZLT4,Kim Cieszykowski,1348531200,1,This coffee is horrible and unfortunately not ...,2
3,#oc-R11O5J5ZVQE25C,B005HG9ESG,Penguin Chick,1346889600,5,This will be the bottle that you grab from the...,3
4,#oc-R12KPBODL2B5ZD,B007OSBEV0,Christopher P. Presta,1348617600,1,I didnt like this coffee. Instead of telling y...,2


In [8]:
display[display['UserId']=='AZY10LLTJ71NX']

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT (*)
80638,AZY10LLTJ71NX,B001ATMQK2,"undertheshrine ""undertheshrine""",1296691200,5,I bought this 6 pack because for the price tha...,5


In [9]:
display ['COUNT (*)'].sum ()

393063

# Exploratory data analysis

## 1. Data deduplication

In [10]:
display = pd.read_sql_query ("""
SELECT *
FROM REVIEWS
WHERE SCORE != 3 AND UserId = "AR5J8UI46CURR"
ORDER BY ProductID
""", con)

In [11]:
display.head ()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [12]:
# Sorting data according to product id in ascending order

sorted_data = filtered_data.sort_values ('ProductId', axis = 0, ascending = True, inplace = False,
                                        kind = 'quicksort', na_position = 'last')

sorted_data.shape

(5000, 10)

In [13]:
# dedupliaction of data

final = sorted_data.drop_duplicates (subset = {"UserId", "ProfileName", "Time", "Text"},
                                    keep = 'first', inplace = False)
final.shape

(4986, 10)

In [14]:
display = pd.read_sql_query ("""
SELECT *
FROM REVIEWS
WHERE Score != 3 AND Id = 44737 or Id = 64422
ORDER BY ProductID
""", con)

display.head ()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [15]:
final  = final [final ["HelpfulnessNumerator"] <= final ["HelpfulnessDenominator"]]
final.shape

(4986, 10)

In [16]:
final ['Score'].value_counts ()

1    4178
0     808
Name: Score, dtype: int64

## 2. Text Preprocessing

In [17]:
final.dtypes

Id                         int64
ProductId                 object
UserId                    object
ProfileName               object
HelpfulnessNumerator       int64
HelpfulnessDenominator     int64
Score                      int64
Time                       int64
Summary                   object
Text                      object
dtype: object

In [18]:
final.head (2)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
2546,2774,B00002NCJC,A196AJHU9EASJN,Alex Chaffee,0,0,1,1282953600,thirty bucks?,Why is this $[...] when the same product is av...
2547,2775,B00002NCJC,A13RRPGE79XFFH,reader48,0,0,1,1281052800,Flies Begone,We have used the Victor fly bait for 3 seasons...


### 1. Removal of HTML tags

In [21]:
sent_0 = final ['Text'].values[0]
print (sent_0)
print ("="*50)

sent_1000 = final ['Text'].values[1000]
print (sent_1000)
print ("="*50)

sent_1500 = final ['Text'].values[1500]
print (sent_1500)
print ("="*50)

sent_4900 = final ['Text'].values[4900]
print (sent_4900)
print ("="*50)

Why is this $[...] when the same product is available for $[...] here?<br />http://www.amazon.com/VICTOR-FLY-MAGNET-BAIT-REFILL/dp/B00004RBDY<br /><br />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.
I recently tried this flavor/brand and was surprised at how delicious these chips are.  The best thing was that there were a lot of "brown" chips in the bsg (my favorite), so I bought some more through amazon and shared with family and friends.  I am a little disappointed that there are not, so far, very many brown chips in these bags, but the flavor is still very good.  I like them better than the yogurt and green onion flavor because they do not seem to be as salty, and the onion flavor is better.  If you haven't eaten Kettle chips before, I recommend that you try a bag before buying bulk.  They are thicker and crunchier than Lays but just as fresh out of the bag.
Wow.  So far, two two-star reviews.  One obviously had no 

In [22]:
# removing urls from python: 

sent_0 = re.sub (r"http\S+", "", sent_0)
sent_1000 = re.sub (r"http\S+", "", sent_1000)
sent_1500 = re.sub (r"http\S+", "", sent_1500)
sent_4900 = re.sub (r"http\S+", "", sent_4900)

print (sent_0)

Why is this $[...] when the same product is available for $[...] here?<br /> /><br />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.


In [23]:
# removing all tags from an element

from bs4 import BeautifulSoup

soup = BeautifulSoup (sent_0, 'html')
text = soup.get_text ()
print (text)
print ("="*50)

soup = BeautifulSoup (sent_1000, 'html')
text = soup.get_text ()
print (text)
print ("="*50)

soup = BeautifulSoup (sent_1500, 'html')
text = soup.get_text ()
print (text)
print ("="*50)

soup = BeautifulSoup (sent_4900, 'html')
text = soup.get_text ()
print (text)
print ("="*50)

Why is this $[...] when the same product is available for $[...] here? />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.
I recently tried this flavor/brand and was surprised at how delicious these chips are.  The best thing was that there were a lot of "brown" chips in the bsg (my favorite), so I bought some more through amazon and shared with family and friends.  I am a little disappointed that there are not, so far, very many brown chips in these bags, but the flavor is still very good.  I like them better than the yogurt and green onion flavor because they do not seem to be as salty, and the onion flavor is better.  If you haven't eaten Kettle chips before, I recommend that you try a bag before buying bulk.  They are thicker and crunchier than Lays but just as fresh out of the bag.
Wow.  So far, two two-star reviews.  One obviously had no idea what they were ordering; the other wants crispy cookies.  Hey, I'm sorry; b

In [24]:
import re

def decontracted (phrase):
    #specific
    phrase = re.sub (r"won't", " will not", phrase)
    phrase = re.sub (r"can\'t", " can not", phrase)
    
    #general
    phrase = re.sub (r"n\'t'", " not", phrase)
    phrase = re.sub (r"\'re'", " are", phrase)
    phrase = re.sub (r"\'s'", " is", phrase)
    phrase = re.sub (r"\'d'", " would", phrase)
    phrase = re.sub (r"\'ll'", " will", phrase)
    phrase = re.sub (r"\'t'", " not", phrase)
    phrase = re.sub (r"\'ve'", " have", phrase)
    phrase = re.sub (r"\'m'", " am", phrase)
    phrase = re.sub (r"aren\'t'", " are not", phrase)
    return phrase

In [25]:
sent_1500 = decontracted (sent_1500)
print (sent_1500)
print ('='*50)

Wow.  So far, two two-star reviews.  One obviously had no idea what they were ordering; the other wants crispy cookies.  Hey, I'm sorry; but these reviews do nobody any good beyond reminding us to look  before ordering.<br /><br />These are chocolate-oatmeal cookies.  If you don't like that combination, don't order this type of cookie.  I find the combo quite nice, really.  The oatmeal sort of "calms" the rich chocolate flavor and gives the cookie sort of a coconut-type consistency.  Now let's also remember that tastes differ; so, I've given my opinion.<br /><br />Then, these are soft, chewy cookies -- as advertised.  They are not "crispy" cookies, or the blurb would say "crispy," rather than "chewy."  I happen to like raw cookie dough; however, I don't see where these taste like raw cookie dough.  Both are soft, however, so is this the confusion?  And, yes, they stick together.  Soft cookies tend to do that.  They aren't individually wrapped, which would add to the cost.  Oh yeah, cho

In [26]:
# removing words with numbers

sent_0 = re.sub ("\S*\d\S*", "", sent_0).strip ()
print (sent_0)

Why is this $[...] when the same product is available for $[...] here?<br /> /><br />The Victor  and  traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.


In [27]:
# remove special character
print (sent_1500)
print ('='*100)
sent_1500 = re.sub ('[^A-Za-z0-9]+', ' ', sent_1500)
print (sent_1500)

Wow.  So far, two two-star reviews.  One obviously had no idea what they were ordering; the other wants crispy cookies.  Hey, I'm sorry; but these reviews do nobody any good beyond reminding us to look  before ordering.<br /><br />These are chocolate-oatmeal cookies.  If you don't like that combination, don't order this type of cookie.  I find the combo quite nice, really.  The oatmeal sort of "calms" the rich chocolate flavor and gives the cookie sort of a coconut-type consistency.  Now let's also remember that tastes differ; so, I've given my opinion.<br /><br />Then, these are soft, chewy cookies -- as advertised.  They are not "crispy" cookies, or the blurb would say "crispy," rather than "chewy."  I happen to like raw cookie dough; however, I don't see where these taste like raw cookie dough.  Both are soft, however, so is this the confusion?  And, yes, they stick together.  Soft cookies tend to do that.  They aren't individually wrapped, which would add to the cost.  Oh yeah, cho

In [28]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
# <br /><br /> ==> after the above steps, we are getting "br br"
# we are including them into stop words list
# instead of <br /> if we have <br/> these tags would have revmoved in the 1st step

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [29]:
# combining all above for students

from tqdm import tqdm
preprocessed_reviews = []
#tqdm is for printing the status bar
for sentance in tqdm(final['Text'].values):
    sentance = re.sub (r"http\S+", '', sentance)
    sentance = BeautifulSoup (sentance, 'html').get_text ()
    sentance = decontracted (sentance)
    sentance = re.sub ('\S*\d\S*', '', sentance).strip ()
    sentance = re.sub ('[^A-Za-z0-9]+', ' ', sentance)
    sentance = ' '.join (e.lower () for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_reviews.append (sentance.strip ())

100%|████████████████████████████████████████████████████████████████████████████| 4986/4986 [00:01<00:00, 4197.59it/s]


In [30]:
preprocessed_reviews [1500]

'wow far two two star reviews one obviously no idea ordering wants crispy cookies hey sorry reviews nobody good beyond reminding us look ordering chocolate oatmeal cookies like combination order type cookie find combo quite nice really oatmeal sort calms rich chocolate flavor gives cookie sort coconut type consistency let also remember tastes differ given opinion soft chewy cookies advertised not crispy cookies blurb would say crispy rather chewy happen like raw cookie dough however see taste like raw cookie dough soft however confusion yes stick together soft cookies tend individually wrapped would add cost oh yeah chocolate chip cookies tend somewhat sweet want something hard crisp suggest nabiso ginger snaps want cookie soft chewy tastes like combination chocolate oatmeal give try place second order'

# Preporcess Summary

# Featurization

## Bagging of words

In [34]:
#BOW
count_vect = CountVectorizer ()
count_vect.fit (preprocessed_reviews)
print ("Some features name: ", count_vect.get_feature_names ()[:10])
print ('='*50)

final_counts = count_vect.transform (preprocessed_reviews)
print ("the type of count vectorizer ", type (final_counts))
print ("The shape of our text BOW vectorizer ", final_counts.get_shape ())
print ("The number of unique words ", final_counts.get_shape ()[1])

Some features name:  ['aa', 'aahhhs', 'aback', 'abandon', 'abates', 'abbott', 'abby', 'abdominal', 'abiding', 'ability']
the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
The shape of our text BOW vectorizer  (4986, 13027)
The number of unique words  13027


# [4.2] Bi-gram and n_grams

In [40]:
count_vect = CountVectorizer (ngram_range = (1, 2), min_df = 10, max_features = 5000)
final_bigram_counts = count_vect.fit_transform (preprocessed_reviews)
print ("Some features name: ", count_vect.get_feature_names ()[:10])
print ('='*50)

print ("the type of count vectorizer ", type (final_bigram_counts))
print ("The shape of our text BOW vectorizer ", final_bigram_counts.get_shape ())
print ("The number of words including both unigrams and bigrams are ", final_bigram_counts.get_shape ()[1])

Some features name:  ['ability', 'able', 'able find', 'able get', 'absolute', 'absolutely', 'absolutely delicious', 'absolutely love', 'absolutely no', 'according']
the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
The shape of our text BOW vectorizer  (4986, 3086)
The number of words including both unigrams and bigrams are  3086


## [4.3] TF-IDF Vectorizer

In [43]:
tf_idf_vect = TfidfVectorizer (ngram_range = (1,2), min_df = 10)
tf_idf_vect.fit (preprocessed_reviews)
print ("some sample features (uniquewords in the corpus)", tf_idf_vect.get_feature_names ()[0:10])
print ('='*50)

final_tf_idf = tf_idf_vect.transform (preprocessed_reviews)
print ("The type of count vetorizer ", type (final_tf_idf))
print ("The shape of our text TFIDF vectorizer ", final_tf_idf.get_shape ())
print ("The number of unique words including both unigram and bigrams ", final_tf_idf.get_shape ()[1])


some sample features (uniquewords in the corpus) ['ability', 'able', 'able find', 'able get', 'absolute', 'absolutely', 'absolutely delicious', 'absolutely love', 'absolutely no', 'according']
The type of count vetorizer  <class 'scipy.sparse.csr.csr_matrix'>
The shape of our text TFIDF vectorizer  (4986, 3086)
The number of unique words including both unigram and bigrams  3086


# [4.2] Word2Vec